## Question Answering su modelli in locale

Implementazione di una chain RAG completamente in locale, con le seguenti features:
* usando Ollama - Nomic - ChromaDB - Gemma
* decomposizione della domanda utente
* sintesi del contesto

# Google Gemma

<img src="https://i2.res.24o.it/images2010/Editrice/ILSOLE24ORE/NOVA24/2024/02/23/Nova24/ImmaginiWeb/Ritagli/Gemma_blog_2-kfUG--1020x533@IlSole24Ore-Web.jpg?r=1080x566" width=400>  
  
Google Gemma è una famiglia di modelli aperti e leggeri, basati sulla tecnologia usata per creare i modelli Google Gemini.  
Sono modelli decoder-only e sono adatti per una varietà di task che vanno dalla generazione di testo, tra cui la risposta alle domande e il riepilogo fino al ragionamento. Le loro dimensioni relativamente ridotte ne consentono l’implementazione in ambienti con risorse limitate come laptop, desktop o la propria infrastruttura cloud, democratizzando l’accesso a modelli di intelligenza artificiale all’avanguardia.  
Le prime versioni della famiglia Gemma si aspettano input testuali sotto forma di domanda o di testo da riassumere e sono state addestreate a rispondere solo in lingua inglese.
([elenco versioni Google Gemma](https://ai.google.dev/gemma/docs/releases))      
Addestrati su un set di dati di 6 trilioni di token da web, codice e testi matematici, questi modelli acquisiscono una vasta conoscenza di stili linguistici, vocaboli, sintassi di programmazione e ragionamento logico, risultando potenti strumenti per diverse attività e formati di testo.  
Gemma è stata addestrata utilizzando i processori TPU custom di Google per sfruttare al meglio prestazioni, memoria, scalabilità ed efficienza in termini di costi, in linea con l'impegno di Google per la sostenibilità.

Questa famiglia di modelli è costruita sulla stessa ricerca e tecnologia che ha dato vita alla famiglia Gemini di Google, ma con un focus diverso: accessibilità, efficienza, usabilità su hardware più modesto.

Nelle versioni più recenti (Gemma 3) si trovano supporti per oltre 140 lingue, contesti molto estesi, e anche input multimodali (testo + immagine).

In [1]:
from typing import TypedDict
# caricamento del modello

from langchain_ollama import ChatOllama

model = ChatOllama(model="gemma3:12b", num_ctx=4096, num_predict=4096)

In [2]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

url = "https://datamasters.it/"
loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text.replace("\n", " ").replace("\t", " ").strip()
)
docs = loader.load()

In [3]:
len(docs)

45

In [4]:
docs[0].metadata

{'source': 'https://datamasters.it/',
 'content_type': 'text/html',
 'title': 'I migliori Corsi di Intelligenza Artificiale | Data Masters',
 'description': 'I migliori corsi online di intelligenza artificiale, machine learning e data science. Formazione pratica, docenti esperti, lezioni on-demand e live. Entra ora!',
 'language': 'it-IT'}

In [5]:
len(docs[0].page_content)

12079

In [6]:
docs[0].page_content

"I migliori Corsi di Intelligenza Artificiale | Data Masters                                                                CREA AGENTI AI ULTIMI POSTI AL 50%\xa0 Scopri di piùDataMastersCatalogoCorsiPercorsi di CarrieraNon sai da dove partire? Compila il test di orientamento e ricevi consigli personalizzati Read morePrincipianteIntermedioAvanzatoCategorieCorsi di Generative AICorsi di Data AnalysisCorsi di Data ScienceCorsi di Machine LearningCorsi di Programmazione in PythonTutti i corsiPer le aziendeCommunityCommunity HubBlogChallengesWebinarsAI e Data Skill Report 2025Chi siamoLavora con noiContattiAccediFai crescere la tua carriera. Inizia davvero a usare dati e AI Formazione pratica e flessibile su Intelligenza Artificiale, Data Science, Machine Learning e Generative AI.Corsi live e on-demand progettati per far evolvere le tue competenze, con la guida di docenti esperti. Scopri il nostro catalogo  Trustpilot   Formiamo i professionisti delle migliori aziende al mondo  Novità  Per

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=250)
splits = text_splitter.split_documents(docs)

In [8]:
print("Numero di chunks : ",len(splits))

Numero di chunks :  828


In [9]:
print("Metadata :",splits[0].metadata )
print ("Dimensione Chunk n.1:",len (splits[0].page_content), " caratteri")
print ("Contenuto Chunk n.1 : \n")
print(splits[0].page_content)

Metadata : {'source': 'https://datamasters.it/', 'content_type': 'text/html', 'title': 'I migliori Corsi di Intelligenza Artificiale | Data Masters', 'description': 'I migliori corsi online di intelligenza artificiale, machine learning e data science. Formazione pratica, docenti esperti, lezioni on-demand e live. Entra ora!', 'language': 'it-IT'}
Dimensione Chunk n.1: 1197  caratteri
Contenuto Chunk n.1 : 

I migliori Corsi di Intelligenza Artificiale | Data Masters                                                                CREA AGENTI AI ULTIMI POSTI AL 50%  Scopri di piùDataMastersCatalogoCorsiPercorsi di CarrieraNon sai da dove partire? Compila il test di orientamento e ricevi consigli personalizzati Read morePrincipianteIntermedioAvanzatoCategorieCorsi di Generative AICorsi di Data AnalysisCorsi di Data ScienceCorsi di Machine LearningCorsi di Programmazione in PythonTutti i corsiPer le aziendeCommunityCommunity HubBlogChallengesWebinarsAI e Data Skill Report 2025Chi siamoLavor

In [11]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
import os

# Creazione di un database vettoriale

if os.path.isdir("data/vs_backup"):
    vector_store = Chroma(embedding_function=OllamaEmbeddings(model="nomic-embed-text"), persist_directory= "data/backup", collection_name="dm_website")
else:
    vector_store = Chroma.from_documents(documents=splits, embedding=OllamaEmbeddings(model="nomic-embed-text"), persist_directory="data/backup", collection_name="dm_website")

In [12]:
# come cercare nel VectorStore

docs = vector_store.similarity_search("Starter Kit", k=3)

for doc in docs:
    print ("documento:", doc.metadata["source"]) 
    print (doc.page_content)
    print ("_" * 80, "\n")

In [13]:
# usiamo il database vettoriale creato poco fa come
# elemento "retriever" per la nostra catena di
# generazione basata su RAG

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10}
)

### Query-decomposition

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

class QuestionReformulator(BaseModel):
    """Reformulate a question into multiple sub-question"""
    sub_questions: List[str] = Field(description="focused sub-question")

decompose_query_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """You are an experienced reasoning assistant, specialized in breaking down user queries into multiple targeted sub-questions for an Augmented Generation Retrieval pipeline.

Your goal is to reformulate the input question into a short list of smaller, precise, and non-overlapping sub-questions that together cover all the aspects needed to answer the original question.
Each sub-question must be self-contained and optimized for semantic search across a document knowledge base.
Avoid redundancy, maintain factual neutrality, and favor specificity over breadth."""),

    ("user",
     "Original question: {question}\n\n"
     "Now generate three sub-questions that will help you gather all the relevant information needed to answer them fully.")
])

sub_questions_generator = RunnableParallel(
    question = RunnablePassthrough(),
    sub_questions = decompose_query_prompt | model.with_structured_output(QuestionReformulator)
)

sub_questions = sub_questions_generator.invoke("quale corso mi consigli per partire da zero?")

In [15]:
print(sub_questions)

{'question': 'quale corso mi consigli per partire da zero?', 'sub_questions': QuestionReformulator(sub_questions=['Quali sono le competenze richieste per iniziare a lavorare nel campo dello sviluppo software?', 'Quali sono i corsi online o in presenza più popolari e ben recensiti per principianti nello sviluppo software in italiano?', 'Quali sono i costi medi e la durata tipica dei corsi per sviluppatori per principianti?'])}


In [16]:
for sub_question in sub_questions["sub_questions"].sub_questions:
    print(sub_question, "\n\n")

Quali sono le competenze richieste per iniziare a lavorare nel campo dello sviluppo software? 


Quali sono i corsi online o in presenza più popolari e ben recensiti per principianti nello sviluppo software in italiano? 


Quali sono i costi medi e la durata tipica dei corsi per sviluppatori per principianti? 




### Sintesi del contesto

In [17]:
summarization_prompt = ChatPromptTemplate.from_messages([
    ("system", """Act like an expert copywriter.
Summarize the following text fragments in a single text, summarizing all the information in an orderly and readable manner.
Do not add preambles or conclusions; simply generate a natural language text that describes all the information."""),
    ("user", "{context}")
])

summarization_chain = summarization_prompt | model | StrOutputParser()

In [ ]:
# {'question': '...', 'sub_questions': QuestionReformulator(sub_questions=['..', '..', '..'])}
def create_context(question_and_subquestions: TypedDict):
    print("# sub questions generation")
    context = []
    for i, sub_question in enumerate(question_and_subquestions["sub_questions"].sub_questions, 1):

        sub_context = "\n\n".join([doc.page_content for doc in retriever.invoke(sub_question)])

        summary = summarization_chain.invoke({"context": sub_context})

        context.append(summary)

        print(i, sub_question)

    context = summarization_chain.invoke({"context": "\n\n".join(context)})
    question_and_subquestions["context"] = context

    return question_and_subquestions

In [20]:
context_generation = RunnableParallel(
    question = RunnablePassthrough(),
    sub_questions = decompose_query_prompt | model.with_structured_output(QuestionReformulator)
) | create_context

question_and_context = context_generation.invoke("quale corso mi consigli per partire da zero?")

# sub questions generation
1 Quali sono le competenze di programmazione richieste per i principianti assoluti?
2 Quali sono le piattaforme online (es. Coursera, Udemy, Codecademy) che offrono corsi introduttivi alla programmazione?
3 Quali sono i linguaggi di programmazione più consigliati per i principianti e perché?


In [21]:
question_and_context

{'question': 'quale corso mi consigli per partire da zero?',
 'sub_questions': QuestionReformulator(sub_questions=['Quali sono le competenze di programmazione richieste per i principianti assoluti?', 'Quali sono le piattaforme online (es. Coursera, Udemy, Codecademy) che offrono corsi introduttivi alla programmazione?', 'Quali sono i linguaggi di programmazione più consigliati per i principianti e perché?']),
 'context': "You're absolutely right to call me out! I got stuck in a loop of offering to summarize. My apologies. I'm ready for the text fragments now. Please provide them!"}

In [20]:
# riscriviamo la funzione senza la print di debug

def create_context(question_and_subquestions: TypedDict):
    context = []
    for i, sub_question in enumerate(question_and_subquestions["sub_questions"].sub_questions):
        sub_context = "\n\n".join([doc.page_content for doc in retriever.invoke(sub_question)])
        summary = summarization_chain.invoke({"context": sub_context})
        context.append(summary)
    context = summarization_chain.invoke({"context": "\n\n".join(context)})
    question_and_subquestions["context"] = context
    return question_and_subquestions

In [22]:
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise."),
    ("user", "Context: {context}\n\nQuestion: {question}\n\nAnswer:")
])

rag_chain = RunnableParallel(
        question = RunnablePassthrough(),
        sub_questions = decompose_query_prompt | model.with_structured_output(QuestionReformulator)
    ) | create_context | rag_prompt | model | StrOutputParser()

In [23]:
rag_chain.invoke("quale corso mi consigli per partire da zero?")

# sub questions generation
1 Quali sono le competenze di base necessarie per iniziare a programmare?
2 Quali sono i linguaggi di programmazione più adatti ai principianti?
3 Quali risorse (corsi online, tutorial, libri) sono raccomandate per imparare a programmare da zero?


'I am still waiting for the text fragments to provide an answer. Please provide the text so I can recommend a course for beginners. Once you provide the text, I will gladly assist you.'

In [24]:
rag_chain.invoke("quali servizi offre Data Masters alle aziende?")

# sub questions generation
1 Quali sono i servizi specifici offerti da Data Masters per la gestione dei dati aziendali?
2 Data Masters offre soluzioni per l'analisi dei dati o business intelligence?
3 Quali sono le aree di competenza o le industrie servite da Data Masters?


"I don't have enough information to answer the question. The provided context only indicates a readiness to receive and summarize text fragments. There is no mention of the services offered by Data Masters."

In [24]:
# DIY
# modificare la chain per avere in output anche la lista delle sotto-domande generate e le relative risposte, nonchè i documenti recuperati e usati come contesto per le risposte